In [646]:
import json
import requests
import numpy as np
from dateutil.relativedelta import relativedelta
from config import api_key, lw_key
import pandas as pd
from datetime import date, timedelta, datetime
import geopy
import pgeocode
nomi = pgeocode.Nominatim('US')


In [377]:
url= "https://api.worldweatheronline.com/premium/v1/weather.ashx?"
past_url = "https://api.worldweatheronline.com/premium/v1/past-weather.ashx?"
# query_url =f"{url}appid={api_key}&zip="

In [17]:
zipcode = input("Enter 5-digit Zipcode:")

Enter 5-digit Zipcode:21230


In [ ]:
# zip_df = nomi.query_postal_code(zipcode)


In [ ]:
# pandas apply pgeocode to ZIPCODE1
# md_zips = df['ZIPCODE1'].apply(nomi.query_postal_code)
# md_zips.head()

In [277]:
response = requests.get(f"{url}key={lw_key}&q={zipcode}&num_of_days=8&tp=24&cc=no&mca=no&aqi=yes&format=json").json()
response = response['data']


In [292]:
weather_dict ={
    "Dates":[],
    "Cloudcover":[],
    "Humidity":[],
    "PrecipInch":[],
    "Pressure":[],
    "FeelsLike":[],
    "HeatIndex":[],
    "MaxTemp":[],
    "MinTemp":[],
    "SunHours":[],
    "UVIndex":[],
}

weather_dict["Dates"]= ([response["weather"][i]["date"] for i in range(len(response["weather"]))])
weather_dict["Cloudcover"]=([response["weather"][i]["hourly"][0]["cloudcover"] for i in range(len(response["weather"]))])
weather_dict["Humidity"]= ([response["weather"][i]["hourly"][0]["humidity"] for i in range(len(response["weather"]))])
weather_dict["PrecipInch"]= ([response["weather"][i]["hourly"][0]["precipInches"] for i in range(len(response["weather"]))])
weather_dict["Pressure"]= ([response["weather"][i]["hourly"][0]["pressure"] for i in range(len(response["weather"]))])
weather_dict["FeelsLike"]= ([response["weather"][i]["hourly"][0]["FeelsLikeF"] for i in range(len(response["weather"]))])
weather_dict["HeatIndex"]= ([response["weather"][i]["hourly"][0]["HeatIndexF"] for i in range(len(response["weather"]))])
weather_dict["MaxTemp"]= ([response["weather"][i]["maxtempF"] for i in range(len(response["weather"]))])
weather_dict["MinTemp"]=([response["weather"][i]["mintempF"] for i in range(len(response["weather"]))])
weather_dict["SunHours"]=([response["weather"][i]["sunHour"] for i in range(len(response["weather"]))])
weather_dict["UVIndex"]= ([response["weather"][i]["hourly"][0]["uvIndex"] for i in range(len(response["weather"]))])
weather_df= pd.DataFrame.from_dict(weather_dict,orient='index').transpose()
weather_df= weather_df.apply(pd.to_numeric, errors= 'ignore')
weather_df["TempDelta"] = weather_df.MaxTemp - weather_df.MinTemp
weather_df["BarChange"] =weather_df["Pressure"].pct_change()
weather_df["HeatChange"] =weather_df["HeatIndex"].pct_change()
weather_df["HumChange"] =weather_df["Humidity"].pct_change()
weather_df= weather_df.iloc[1:]
weather_df.head(10)


,Dates,Cloudcover,Humidity,PrecipInch,Pressure,FeelsLike,HeatIndex,MaxTemp,MinTemp,SunHours,UVIndex,TempDelta,BarChange,HeatChange,HumChange
1,2021-04-15,55,70,0.2,1008,53,55,62,48,9.1,3,14,-0.007874,-0.017857,-0.176471
2,2021-04-16,43,61,0.0,1008,42,48,54,44,7.8,4,10,0.000000,-0.127273,-0.128571
3,2021-04-17,61,65,0.0,1011,46,49,53,44,6.6,4,9,0.002976,0.020833,0.065574
4,2021-04-18,47,62,0.0,1012,50,52,58,45,7.9,1,13,0.000989,0.061224,-0.046154
5,2021-04-19,60,64,0.0,1014,53,54,61,47,6.6,3,14,0.001976,0.038462,0.032258
6,2021-04-20,35,60,0.0,1016,56,57,67,48,10.4,4,19,0.001972,0.055556,-0.062500
7,2021-04-21,37,57,0.0,1014,51,54,58,48,10.4,3,10,-0.001969,-0.052632,-0.050000


In [659]:
dates= input("Dates you had a migraine (YYYY-MM-DD):")

Dates you had a migraine (YYYY-MM-DD):2021-01-03, 2021-02-02, 2021-03-03


In [660]:
dates=dates.split(",")
dates= [i.strip() for i in dates]
dates_df = pd.DataFrame(dates,columns=['Dates'])
dates_df["Migraine"] = 1

In [635]:
history_dict ={
    "Dates":[],
    "Cloudcover":[],
    "Humidity":[],
    "PrecipInch":[],
    "Pressure":[],
    "FeelsLike":[],
    "HeatIndex":[],
    "MaxTemp":[],
    "MinTemp":[],
    "SunHours":[],
    "UVIndex":[],
    }

In [636]:
for i in range(6,0,-1):
    date= (datetime.today()-relativedelta(months=i)).strftime('%Y-%m-%d')
    enddate=(datetime.today()-relativedelta(months=i-1)).strftime('%Y-%m-%d')
    responses = requests.get(f"{past_url}key={lw_key}&q={zipcode}&date={date}&enddate={enddate}&tp=24&mca=no&aqi=yes&format=json").json()
    responses = responses['data']
    history_dict["Dates"].append([responses["weather"][i]["date"] for i in range(len(responses["weather"]))])
    history_dict["Cloudcover"].append([responses["weather"][i]["hourly"][0]["cloudcover"] for i in range(len(responses["weather"]))])
    history_dict["Humidity"].append([responses["weather"][i]["hourly"][0]["humidity"] for i in range(len(responses["weather"]))])
    history_dict["PrecipInch"].append([responses["weather"][i]["hourly"][0]["precipInches"] for i in range(len(responses["weather"]))])
    history_dict["Pressure"].append([responses["weather"][i]["hourly"][0]["pressure"] for i in range(len(responses["weather"]))])
    history_dict["FeelsLike"].append([responses["weather"][i]["hourly"][0]["FeelsLikeF"] for i in range(len(responses["weather"]))])
    history_dict["HeatIndex"].append([responses["weather"][i]["hourly"][0]["HeatIndexF"] for i in range(len(responses["weather"]))])
    history_dict["MaxTemp"].append([responses["weather"][i]["maxtempF"] for i in range(len(responses["weather"]))])
    history_dict["MinTemp"].append([responses["weather"][i]["mintempF"] for i in range(len(responses["weather"]))])
    history_dict["SunHours"].append([responses["weather"][i]["sunHour"] for i in range(len(responses["weather"]))])
    history_dict["UVIndex"].append([responses["weather"][i]["hourly"][0]["uvIndex"] for i in range(len(responses["weather"]))])

In [681]:
history_df= pd.DataFrame.from_dict(history_dict,orient='index').transpose()
history_df= history_df.apply(pd.to_numeric, errors= 'ignore')


In [682]:
history_df= history_df.apply(pd.Series.explode).reset_index(drop=True)
history_df= history_df.apply(pd.to_numeric, errors= 'ignore')
history_df.reset_index(drop=True)
history_df.head()

,Dates,Cloudcover,Humidity,PrecipInch,Pressure,FeelsLike,HeatIndex,MaxTemp,MinTemp,SunHours,UVIndex
0,2020-10-15,6,58,0.0,1015,65,66,74,59,11.1,5
1,2020-10-16,74,59,0.0,1022,56,57,57,54,7.1,3
2,2020-10-17,9,48,0.0,1026,51,53,62,41,11.1,4
3,2020-10-18,2,50,0.0,1029,59,59,63,55,11.0,5
4,2020-10-19,35,76,0.0,1026,61,61,67,56,10.1,4


In [683]:
history_df["TempDelta"]= history_df.MaxTemp - weather_df.MinTemp
history_df["BarChange"]= history_df["Pressure"].pct_change()
history_df["HeatChange"]= history_df["HeatIndex"].pct_change()
history_df["HumChange"]= history_df["Humidity"].pct_change()
history_df= pd.merge(history_df,dates_df, on = 'Dates', how = 'left')
history_df= history_df.fillna(0)
history_df= history_df.iloc[1:]
history_df.head(10)

,Dates,Cloudcover,Humidity,PrecipInch,Pressure,FeelsLike,HeatIndex,MaxTemp,MinTemp,SunHours,UVIndex,TempDelta,BarChange,HeatChange,HumChange,Migraine
1,2020-10-16,74,59,0.0,1022,56,57,57,54,7.1,3,9.0,0.006897,-0.136364,0.017241,0.0
2,2020-10-17,9,48,0.0,1026,51,53,62,41,11.1,4,18.0,0.003914,-0.070175,-0.186441,0.0
3,2020-10-18,2,50,0.0,1029,59,59,63,55,11.0,5,19.0,0.002924,0.113208,0.041667,0.0
4,2020-10-19,35,76,0.0,1026,61,61,67,56,10.1,4,22.0,-0.002915,0.033898,0.520000,0.0
5,2020-10-20,26,74,0.0,1024,65,66,75,59,11.0,5,28.0,-0.001949,0.081967,-0.026316,0.0
6,2020-10-21,58,84,0.0,1024,72,72,77,66,8.5,5,29.0,0.000000,0.090909,0.135135,0.0
7,2020-10-22,6,76,0.0,1023,74,74,80,66,10.9,6,32.0,-0.000977,0.027778,-0.095238,0.0
8,2020-10-23,22,77,0.0,1021,69,71,74,65,10.9,6,0.0,-0.001955,-0.040541,0.013158,0.0
9,2020-10-24,39,72,0.1,1017,68,70,75,64,8.5,5,0.0,-0.003918,-0.014085,-0.064935,0.0
10,2020-10-25,98,72,0.5,1021,47,50,50,46,5.4,2,0.0,0.003933,-0.285714,0.000000,0.0


In [684]:
history_df.to_csv("../../Desktop/test.csv")